In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from Dataset.WindowDataset import WindowDataset

from Model.AE import Auto
from Model.LSTM import LSTM
from Model.LODE2 import ODEVAE

from utils import combine_tensors

In [2]:
eco = '콜금리'
batch_size = 4
max_apart_size = 70  # 0~60평 아파트만 돌림
real_estate_weighted_average = 0.5
input_size = 5 # LSTM input_size
hidden_size = 16 # LSTM hidden_size

In [3]:
def make_estimate(x):
    vec = np.zeros((max_apart_size))
    vec[int(x[0])] = x[1]
    return vec

In [4]:
df = pd.read_csv('../데이터/merge.csv')
df['시점'] = df['계약년월'] + df['계약일']
df = df[:100] # //////////////////////////////
df['시점'].apply(lambda x : int(x))
df['평수'] = df['전용면적(㎡)'].apply(lambda x : x // 3.3058)
df['가격'] = df['거래금액(만원)'].apply(lambda x : int(x.replace(',','')))
df = df.sort_values(by=['시점'])
df['부동산'] = df[['평수','가격']].apply(make_estimate,axis=1)
df.reset_index(drop=True)
df['경제'] = df[eco]
df = df[['시점','부동산','경제']]

In [5]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 2

train_dataset = WindowDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = WindowDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = WindowDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/Users/kimhakhyun/Desktop/Git/sci/SCI/코드/Dataset/WindowDataset.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  self.x1 = torch.FloatTensor(X1)


In [6]:
for x1,x2,y1,y2 in train_loader:
    print(x1.shape)
    print(x2.shape)
    break

torch.Size([2, 5, 2])
torch.Size([2, 5, 1, 70])


In [7]:
######### 모델

model = ODEVAE(2, 64, 6)

In [8]:
#############################
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

# model = Model.to(device)

# print('작동하는지 실험')
# basic_data = torch.rand((1,input_size))
# model(basic_data)

criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 10
noise_std = 0.02
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

cpu is available


In [9]:
economy_embedding_model = LSTM(input_size,hidden_size).to(device)
economy_embedding_model.load_state_dict(torch.load('Checkpoint/economic_best_model.pth'))

real_estate_embedding_model = Auto(max_apart_size).to(device)
real_estate_embedding_model.load_state_dict(torch.load('Checkpoint/embedding_best_model.pth'))

RuntimeError: Error(s) in loading state_dict for Auto:
	size mismatch for encoder.0.weight: copying a param with shape torch.Size([32, 7]) from checkpoint, the shape in current model is torch.Size([32, 70]).
	size mismatch for decoder.2.weight: copying a param with shape torch.Size([7, 32]) from checkpoint, the shape in current model is torch.Size([70, 32]).
	size mismatch for decoder.2.bias: copying a param with shape torch.Size([7]) from checkpoint, the shape in current model is torch.Size([70]).

In [10]:
basic_data = torch.rand((5,input_size))
time_data = [20180103,20180904,20180906,20181011,20190101]

tensor([[0.6234, 0.0333, 0.9214, 0.7519, 0.8305],
        [0.6660, 0.7309, 0.4756, 0.0741, 0.2760],
        [0.7328, 0.8379, 0.7091, 0.5508, 0.4279],
        [0.8611, 0.5838, 0.0804, 0.2475, 0.6017],
        [0.2672, 0.2526, 0.6937, 0.8916, 0.1437]])

In [ ]:
for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for x1, x2, y1, y2 in train_loader:
        x1, x2 = x1.to(device), x2.to(device) # x1: [시점, 경제], x2: 부동산
        y1, y2 = y1.to(device), y2.to(device)

        # 경제 데이터
        eco_emb = economy_embedding_model(x1[1])
        # 부동산 데이터
        real_est_emb = real_estate_embedding_model(x2)
        
        final_emb = combine_tensors(real_est_emb, eco_emb, real_estate_weighted_average, 'sum')
        ##########################
        # 모델에 x[0], final_emb 가지고 ODE 학습하기!!!!

        optimizer.zero_grad()

        x_p, z, z_mean, z_log_var = model(final_emb, x1[0])
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean**2 - torch.exp(z_log_var), -1)
        loss = 0.5 * ((final_emb-x_p)**2).sum(-1).sum(0) / noise_std**2 + kl_loss
        loss = torch.mean(loss)
       
        loss.backward()
        optimizer.step()